Some of these boxes are heavy duty (for the pc) and some of these are just light enough

In [2]:
# machine translation; USE THE PC FOR THIS
from transformers import pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-tl")

print(pipe("This is a job for me."))

Device set to use cpu


[{'translation_text': "Ito'y isang trabaho para sa akin."}]


In [14]:
import tika
from tika import parser
import os

print(os.listdir("./booksEpub"))

epubList = []

for i in os.listdir("./booksEpub"):
    filePath = "./booksEpub/" + i
    epubList.append(parser.from_file(filePath))

['Ang_Bangka_ni_Tatay_Mario___Filipino-v12023.04.12T012033.epub', 'Ang_Paglalaro_nina_Mina_at_Manay___Filipino-v12023.06.16T022744.epub', 'Ano_ang_Kinatatakutan_nina__Anopol_at_ni_Tang-id___Filipino-v12023.06.20T194135.epub', 'Luna_Papelastika___Filipino-v12023.04.07T035616.epub', 'Misteryo_ng_Itlog___Filipino.epub', 'Pwedeng-pwede,_Pani!___Filipino-v12023.04.07T033002.epub', 'Sino_ang_Magiging_Titser_Ko_sa_Pasukan____Filipino-v12023.11.14T034013.epub', 'Si_Kokoy_at_ang_Umang-umang___Filipino-v12023.04.07T084156.epub', 'Si_Pogi,_Si_Pogs_At_Ang_Tora-tora___Filipino-v12023.11.14T082955.epub', 'Super_Wanda___Filipino-v12023.04.07T025901.epub']


In [ ]:
import pymupdf
import os

print(os.listdir("./booksEpub"))

ebookContent = {}

for i in os.listdir("./booksEpub"):
    ebookText = ""
    titlePageText = ""
    filePath = "./booksEpub/" + i
    doc = pymupdf.open(filePath)
    for chapter in range(doc.chapter_count):
        print(f"Chapter {chapter}")
        if chapter==0:
            # get the texts and it will be the key
            currentPage = doc[(0,0)]
            titlePageText = currentPage.get_text()
        else:
            for page in range(doc.chapter_page_count(chapter)):
                print(f"page {page}")
                currentPage = doc[(chapter, page)]
                currentText = currentPage.get_text()
                print(currentText)
                # check if it's the last few pages
                if not(("Brought to you by" in currentText) or ("International License" in currentText) or ("Table of Contents" in currentText) or ("This book was developed as part of the ABC" in currentText) or ("Advancing Basic Education in the Philippines" in currentText)):
                    ebookText+=currentText
                else:
                    pass
    
    #print(ebookText)
    ebookContent[titlePageText] = ebookText




In [ ]:
import re

for key in ebookContent.keys():
    print(key)
    processedKey = re.sub(r'[\.]','',key)
    processedKey = re.sub(r' ','_',processedKey)
    processedKey = re.sub(r'\?','',processedKey)
    processedKey = re.sub(r'\n','_',processedKey)
    f = open(f"./textData1_pymupdf/{processedKey}.txt","w",encoding='utf-8')
    #print(ebookContent[key])
    f.writelines(ebookContent[key])
    f.close()
    #print("DONE")

In [10]:
import os
import re

sourceFolder = "./textData1_pymupdf"

totalText = ""

# format the text correctly first
for i in os.listdir(sourceFolder):
    filePathForTokenizingText = f"{sourceFolder}/{i}"
    f = open(filePathForTokenizingText,'r', encoding='utf-8')
    insideText = f.read()
    f.close()

    #formatting with regex

    insideText = re.sub(r'[.]','. ',insideText)
    insideText = re.sub('\n',' ',insideText)
    insideText = re.sub('- ','-',insideText)
    insideText = re.sub('  ',' ',insideText)
    insideText = re.sub('&nbsp;','',insideText)

    totalText += insideText
    totalText += "\n\n"

f2 = open("./textData_forTokenization/textAll.txt", "w", encoding='utf-8')
f2.writelines(totalText)
f2.close()

    

In [33]:
# organize it properly
import re

tagged_Text = open("./textData_postagged/textAll_tagged.txt", 'r', encoding="utf-16")

allText = tagged_Text.read()
#print(allText)
allText = re.sub(r'\|PMP \"\|PMS\n', '|PMP  \n\"|PMS ', allText)
allText = re.sub(r'\|PMQ \"\|PMS\n', '|PMQ  \n\"|PMS ', allText)
allText = re.sub(r'\|PME \"\|PMS\n', '|PME  \n\"|PMS ', allText)
allText = re.sub(r'\|PMS \"\|PMS\n', '|PMS  \n\"|PMS ', allText)
allText = re.sub(r'\|PMC \"\|PMS\n', '|PMC  \n\"|PMS ', allText)
allText = re.sub(r'\|PMSC \"\|PMS\n','|PMSC \n\"|PMS ', allText)

#print(allText)

final_Text = open("./textData_postagged/textAll_tagged_final.txt", 'w')
final_Text.write(allText)


131162

making the vocabulary and the tokens

In [35]:
import os

postagged_filepath = "./textData_postagged/textAll_tagged_final.txt"

vocabulary = set()

f = open(f"{postagged_filepath}", "r", encoding="utf-8")
story = f.read()
    
for sentence in story.split('\n'):
    for token in sentence.split(' '):
        vocabulary.add(token)

In [ ]:
import pandas as pd

vocabulary_F = vocabulary.remove('')

vocab_df = pd.DataFrame(list(vocabulary), columns=['tokens'])
vocab_df['lemma'] = vocab_df['tokens'].apply(lambda x: x.split('|')[0])
vocab_df['postag'] = vocab_df['tokens'].apply(lambda x: x.split('|')[1])
vocab_df

In [39]:
pd.set_option('display.max_rows', None)
print(vocab_df.loc[ vocab_df['postag']=="NNC" ]['tokens'].shape)
vocab_df.loc[ vocab_df['postag']=="NNC" ]['tokens']

(618,)


0                kapatid|NNC
2               simangot|NNC
6                pintuan|NNC
11                  baso|NNC
16                  usok|NNC
20               kutsara|NNC
24               krayola|NNC
25               paligid|NNC
29                bulong|NNC
30                 bunga|NNC
34              paghinga|NNC
44                lupain|NNC
45                    Pa|NNC
52                balita|NNC
59               sandali|NNC
62               balyena|NNC
66              senyales|NNC
67                 grado|NNC
71                 tanim|NNC
75                  ulap|NNC
86                kusina|NNC
90              meryenda|NNC
91                 niyog|NNC
99                 plano|NNC
103               kamote|NNC
104                baboy|NNC
115                dahan|NNC
116                basag|NNC
119             kamoteng|NNC
122             kaharian|NNC
135               tanong|NNC
139               aalala|NNC
147               tansan|NNC
153               regalo|NNC
154           

In [43]:
# get the vocabulary and the sentences into the Organized_Text_data folder in the form of csv's

import pandas as pd

#vocab_df.to_csv("./Organized_Text_data/vocabulary.csv")

sentenceInp = open("./textData_postagged/textAll_tagged_final.txt", 'r', encoding='utf-8')
sentenceDF = pd.DataFrame(sentenceInp.readlines(), columns=['sentence'])
sentenceDF.to_csv("./Organized_Text_data/sentences.csv")

### The machine learning part after preprocessing

@inproceedings{kuran2024hybrid,
  title={A HYBRID BILSTM-CRF MODEL WITH BERT EMBEDDINGS FOR NAMED ENTITY RECOGNITION ON CONFIDENTIAL SYNTHETIC DATA},
  author={Kuran, Emre Can and Elen, Abdullah and Kuran, Umut},
  booktitle={V. INTERNATIONAL SCIENCE AND INNOVATION CONGRESS (INSI 2024) PROCEEDINGS BOOK},
  pages={16}
}

pre-training the ELMO model so I have the weights needed (source: https://huggingface.co/datasets/linkanjarad/Wikitext-TL39/tree/main)

In [2]:
# convert parquet to text
import pandas as pd
import fastparquet
import pyarrow
import numpy as np

wikiData = pd.read_parquet('./Pre-training/train.parquet')
wikiData.to_csv('./Pre-training/train.txt', header=None, index=None, sep='\n', mode='a')

In [11]:
# clean up the pre-training data
import re

readTrain = open('./Pre-training/train.txt', 'r', encoding='utf-8')
writeThis = re.sub(r"= [a-zA-Z ]+ =", '',readTrain.read())
writeTrain = open('./Pre-training/train_1.txt', 'w', encoding='utf-8')
writeTrain.write(writeThis)

208798244

In [18]:
import re
import pandas as pd

readTrain = open('./Pre-training/train.txt', 'r', encoding='utf-8')
allTopics = re.findall(r"= [a-zA-Z ]+ =", readTrain.read())
allTopicsFinal = []
for i in allTopics:
    allTopicsFinal.append(i[2:-2])

pd.DataFrame(allTopicsFinal, columns=['topics']).to_csv("./Organized_Text_data/vocabulary_wiki_tl39.csv")

In [2]:
# compare vocabularies
import pandas as pd

filwordnet_lemmas = pd.read_csv("./Organized_Text_data/filWordNetLemmas.csv")['lemma'].to_frame()
preTrain_topics = pd.read_csv("./Organized_Text_data/vocabulary_wiki_tl39.csv")['topics'].to_frame()
childrensBook_vocabulary = pd.read_csv("./Organized_Text_data/vocabulary.csv")['lemma'].to_frame()

filwordnet_lemmas.columns = ['vocabulary']
preTrain_topics.columns = ['vocabulary']
childrensBook_vocabulary.columns = ['vocabulary']

filwordnet_lemmas['vocabulary'] = filwordnet_lemmas['vocabulary'].str.lower()
preTrain_topics['vocabulary'] = preTrain_topics['vocabulary'].str.lower()
childrensBook_vocabulary['vocabulary'] = childrensBook_vocabulary['vocabulary'].str.lower()

#print(f"{filwordnet_lemmas.info()} {preTrain_vocabulary.info()} {childrensBook_vocabulary.info()}")

filwordnet_preTrain = filwordnet_lemmas.merge(preTrain_topics, how='inner', on=['vocabulary']).drop_duplicates()
filwordnet_childrensBook = filwordnet_lemmas.merge(childrensBook_vocabulary, how='inner', on=['vocabulary']).drop_duplicates()
preTrain_childrensBook = preTrain_topics.merge(childrensBook_vocabulary, how='inner', on=['vocabulary']).drop_duplicates()

allmerge = filwordnet_preTrain.merge(preTrain_childrensBook, how='inner', on=['vocabulary']).drop_duplicates()

print(f'''
Length of the intersection of each dataset=======================
filwordnet_preTrain: {filwordnet_preTrain.shape[0]}
filwordnet_childrensBook: {filwordnet_childrensBook.shape[0]}
preTrain_childrensBook: {preTrain_childrensBook.shape[0]}
AllDatasets: {allmerge.shape[0]}

''')



Length of the intersection of each dataset=======================
filwordnet_preTrain: 1651
filwordnet_childrensBook: 364
preTrain_childrensBook: 292
AllDatasets: 149




In [24]:
# topic finder sa wiki
import pandas as pd
import re
import json

topicList = pd.concat((filwordnet_preTrain, preTrain_childrensBook)).drop_duplicates()['vocabulary'].str.lower().tolist()
topicSet = set(topicList)

readTrain = open('./Pre-training/train.txt', 'r', encoding='utf-8')
readTrainText = readTrain.readlines()
readTrainText

topicDict_withExamples = {}
currentTopic = "DELETE_THIS_KEY"

for line in readTrainText:
    #print(line, currentTopic)
    if(line[0] == '='):
        # check if the thing we're looking at is at this list
        toCheck = line[2:-3]
        if toCheck.lower() in topicSet:
            currentTopic = toCheck
            #print(currentTopic)
            topicDict_withExamples[currentTopic] = []
        else:
            currentTopic = "DELETE_THIS_KEY"
    else:
        if currentTopic != "DELETE_THIS_KEY":
            topicDict_withExamples[currentTopic].append(line)
        else:
            pass

with open("./Organized_Text_data/topic_withExamples.json", "w") as outfile: 
    json.dump(topicDict_withExamples, outfile)

In [15]:
# merge all sentences in the common topic thing into a single text file
import json
import math as m

commonTopics = open("./Organized_Text_data/topic_withExamples.json")
data = json.load(commonTopics)

keyChunks = []
chunk_count = 4
keylist = list(data.keys())

chunk_size = m.ceil(len(keylist)/chunk_count)

for i in range(chunk_count):
    keyChunks.append(keylist[(chunk_size*i): (chunk_size*(i+1))])

chunk_ind = 1
for chunk in keyChunks:
    sentenceList = []

    for key in chunk:
        for sentence in data[key]:
            sentenceList.append(sentence)

    sentenceList_file = open(f"./Pre-training/train_common_lite_{chunk_ind}.txt", "w", encoding='utf-8')
    sentenceList_file.writelines(sentenceList)
    sentenceList_file.close()
    chunk_ind+=1


In [19]:
# merge the text files into one

preTrainCorpus = open("./Pre-training/train_common_postagged.txt", 'w', encoding='utf-8')

for i in range(1,4+1):
    f = open(f"./Pre-training/train_1_common_lite_{i}.txt", 'r', encoding='utf-16')
    preTrainCorpus.write(f.read())
    f.close()

preTrainCorpus.close()


In [21]:
# format the postag file
# organize it properly
import re

tagged_Text = open("./Pre-training/train_common_postagged.txt", 'r', encoding="utf-8")

allText = tagged_Text.read()
#print(allText)
allText = re.sub(r'\|PMP \"\|PMS\n', '|PMP  \n\"|PMS ', allText)
allText = re.sub(r'\|PMQ \"\|PMS\n', '|PMQ  \n\"|PMS ', allText)
allText = re.sub(r'\|PME \"\|PMS\n', '|PME  \n\"|PMS ', allText)
allText = re.sub(r'\|PMS \"\|PMS\n', '|PMS  \n\"|PMS ', allText)
allText = re.sub(r'\|PMC \"\|PMS\n', '|PMC  \n\"|PMS ', allText)
allText = re.sub(r'\|PMSC \"\|PMS\n','|PMSC \n\"|PMS ', allText)

final_Text = open("./Pre-training/train_1_common_postagged.txt", 'w')
final_Text.write(allText)



5640685

In [77]:
# split the data 80-20
import pandas as pd

trainingData_common_file = open("./Pre-training/train_1_common_postagged.txt", 'r', encoding='utf-8')

trainingData_common = pd.DataFrame(trainingData_common_file.readlines(), columns=['sentences'])

trainingData_common_test = trainingData_common.sample(frac=0.2, random_state=6423)

# get the rows that are in trainingData_common but not it the trainingData_common_test
allddd = pd.DataFrame()
allddd["dd"] = trainingData_common
allddd["qq"] = trainingData_common_test
allddd["final"] = allddd["dd"] * (allddd["dd"] != allddd["qq"])
ffiii = allddd['final'].values.tolist()

pop_index = 0
while pop_index < len(ffiii):
    if ffiii[pop_index] == '':
        ffiii.pop(pop_index)
    else:
        pop_index+=1

trainingData_common_train = pd.DataFrame(ffiii, columns=['sentences'])
print(f"Train: {trainingData_common_train.shape[0]}\nTest: {trainingData_common_test.shape[0]}")



tr = open("./Pre-training/trainingData_ELMO.txt",'w', encoding='utf-8')
tr.writelines(i[0] for i in trainingData_common_train.values.tolist())
tr.close()

te = open("./Pre-training/testData_ELMO.txt",'w', encoding='utf-8')
te.writelines(i[0] for i in trainingData_common_test.values.tolist())
te.close()

Train: 16483
Test: 4121


In [ ]:
# ELMO -> BiLSTM -> CRF

import spacy
from spacy.language import Language
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch

# Load ELMo model
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, num_output_representations=1, dropout=0)

# Define a custom component for ELMo embeddings
@Language.component("elmo_embedder")
def elmo_embedder(doc):
    # Convert SpaCy tokens to ELMo character IDs
    token_ids = batch_to_ids([[token.text for token in doc]])
    
    # Get ELMo embeddings (shape: [batch_size, max_length, 1024])
    embeddings = elmo(token_ids)["elmo_representations"][0]
    
    # Store ELMo embeddings in Doc object
    doc._.elmo_embeddings = embeddings.squeeze(0)  # Remove batch dimension
    return doc